In [1]:
import pandas as pd

In [2]:
df = pd.read_csv('Data/Sentiment/bitcoin/bitcoin_submission_sentiment.csv')
df.head()

,submission,author,num_comments,shortlink,link_flair_text,title,selftext,score,subreddit,posted_on,label,score.1
0,rt711v,Squigglywiggler,9,https://redd.it/rt711v,NaN,When sovereign wealth funds?,Anyone heard any news about this? Any opinions...,12,bitcoin,2022-01-01 00:17:38,Neutral,0.669064
1,rt7qa5,SilverAndAlgorand,510,https://redd.it/rt7qa5,NaN,You have already won.,I only have .03 btc but I still feel early as ...,1176,bitcoin,2022-01-01 00:55:55,Positive,0.655485
2,rt7x3t,Geoknysis,10,https://redd.it/rt7x3t,NaN,Good Android Wallet for BTC,"Okay,\n\nSo I want to move away from Exodus an...",2,bitcoin,2022-01-01 01:06:06,Positive,0.618921
3,rt86pj,Mannycass,6,https://redd.it/rt86pj,NaN,George Athanassakos (Seat tight and HODL),https://www.ivey.uwo.ca/faculty/directory/geor...,6,bitcoin,2022-01-01 01:20:37,Neutral,0.895270
4,rta3p4,Still_Hat6758,11,https://redd.it/rta3p4,NaN,Factored in satoshi Nakamoto’s bitcoins into t...,"Just a genuine question, I’ve googled biggest ...",0,bitcoin,2022-01-01 03:07:40,Neutral,0.578329


In [3]:
# Convert 'posted_on' to datetime
df['posted_on'] = pd.to_datetime(df['posted_on'])

# Set the 'posted_on' as the index (optional)
df.set_index('posted_on', inplace=True)

In [4]:
# Resample or group by a time period (e.g., daily) and count sentiment labels
# Here 'D' stands for daily. You can change it to 'H' for hourly, 'M' for monthly, etc.
time_series = df.resample('D')['label'].value_counts().unstack()
time_series.columns.name = None

In [5]:
time_series

,Negative,Neutral,Positive
posted_on,,,
2022-01-01,5.0,27.0,24.0
2022-01-02,3.0,24.0,22.0
2022-01-03,7.0,37.0,35.0
2022-01-04,4.0,22.0,17.0
2022-01-05,6.0,48.0,26.0
...,...,...,...
2022-12-27,1.0,22.0,5.0
2022-12-28,6.0,30.0,18.0
2022-12-29,2.0,25.0,17.0


In [6]:
# Resetting index to make 'posted_on' a column again (optional)
time_series.reset_index(inplace=True)

# Your new DataFrame 'time_series' is ready
display(time_series)

,posted_on,Negative,Neutral,Positive
0,2022-01-01,5.0,27.0,24.0
1,2022-01-02,3.0,24.0,22.0
2,2022-01-03,7.0,37.0,35.0
3,2022-01-04,4.0,22.0,17.0
4,2022-01-05,6.0,48.0,26.0
...,...,...,...,...
360,2022-12-27,1.0,22.0,5.0
361,2022-12-28,6.0,30.0,18.0
362,2022-12-29,2.0,25.0,17.0
363,2022-12-30,3.0,25.0,18.0


In [7]:
time_series.isna().sum()

posted_on     0
Negative     30
Neutral       0
Positive      0
dtype: int64

In [8]:
time_series.fillna(value=0,inplace=True)

In [9]:
time_series.isna().sum()

posted_on    0
Negative     0
Neutral      0
Positive     0
dtype: int64

In [10]:
time_series['total_posts'] = time_series['Positive']+time_series['Negative']+time_series['Neutral']
time_series

,posted_on,Negative,Neutral,Positive,total_posts
0,2022-01-01,5.0,27.0,24.0,56.0
1,2022-01-02,3.0,24.0,22.0,49.0
2,2022-01-03,7.0,37.0,35.0,79.0
3,2022-01-04,4.0,22.0,17.0,43.0
4,2022-01-05,6.0,48.0,26.0,80.0
...,...,...,...,...,...
360,2022-12-27,1.0,22.0,5.0,28.0
361,2022-12-28,6.0,30.0,18.0,54.0
362,2022-12-29,2.0,25.0,17.0,44.0
363,2022-12-30,3.0,25.0,18.0,46.0


In [11]:
#Calculating sentiment proportions
time_series['Positive'] = round((time_series['Positive']/time_series['total_posts'])*100,2)
time_series['Negative'] = round((time_series['Negative']/time_series['total_posts'])*100,2)
time_series['Neutral'] = round((time_series['Neutral']/time_series['total_posts'])*100,2)
time_series

,posted_on,Negative,Neutral,Positive,total_posts
0,2022-01-01,8.93,48.21,42.86,56.0
1,2022-01-02,6.12,48.98,44.90,49.0
2,2022-01-03,8.86,46.84,44.30,79.0
3,2022-01-04,9.30,51.16,39.53,43.0
4,2022-01-05,7.50,60.00,32.50,80.0
...,...,...,...,...,...
360,2022-12-27,3.57,78.57,17.86,28.0
361,2022-12-28,11.11,55.56,33.33,54.0
362,2022-12-29,4.55,56.82,38.64,44.0
363,2022-12-30,6.52,54.35,39.13,46.0


### Saving the Time Series

In [12]:
time_series_positive = time_series[['posted_on','Positive']]
display(time_series_positive)
time_series_positive.to_csv("Data/Time_Series/bitcoin/bitcoin_sentiment_positive.csv",index=False)

,posted_on,Positive
0,2022-01-01,42.86
1,2022-01-02,44.90
2,2022-01-03,44.30
3,2022-01-04,39.53
4,2022-01-05,32.50
...,...,...
360,2022-12-27,17.86
361,2022-12-28,33.33
362,2022-12-29,38.64
363,2022-12-30,39.13


In [13]:
time_series_negative = time_series[['posted_on','Negative']]
display(time_series_negative)
time_series_negative.to_csv("Data/Time_Series/bitcoin/bitcoin_sentiment_negative.csv",index=False)

,posted_on,Negative
0,2022-01-01,8.93
1,2022-01-02,6.12
2,2022-01-03,8.86
3,2022-01-04,9.30
4,2022-01-05,7.50
...,...,...
360,2022-12-27,3.57
361,2022-12-28,11.11
362,2022-12-29,4.55
363,2022-12-30,6.52


In [14]:
time_series_neutral = time_series[['posted_on','Neutral']]
display(time_series_neutral)
time_series_neutral.to_csv("Data/Time_Series/bitcoin/bitcoin_sentiment_neutral.csv",index=False)

,posted_on,Neutral
0,2022-01-01,48.21
1,2022-01-02,48.98
2,2022-01-03,46.84
3,2022-01-04,51.16
4,2022-01-05,60.00
...,...,...
360,2022-12-27,78.57
361,2022-12-28,55.56
362,2022-12-29,56.82
363,2022-12-30,54.35


In [15]:
time_series_size = time_series[['posted_on','total_posts']]
display(time_series_size)
time_series_size.to_csv("Data/Time_Series/bitcoin/bitcoin_posts.csv",index=False)

,posted_on,total_posts
0,2022-01-01,56.0
1,2022-01-02,49.0
2,2022-01-03,79.0
3,2022-01-04,43.0
4,2022-01-05,80.0
...,...,...
360,2022-12-27,28.0
361,2022-12-28,54.0
362,2022-12-29,44.0
363,2022-12-30,46.0
